In [12]:
import os
os.chdir("/Users/victormanuelfreixaslemus/Desktop/Projects/Git_portafolio/trXPES/tr-XPES")
from src.calc_DO_single_state import calc_DO_single_state
import numpy as np

masterDataFileName = "examples/HBQ/tr_001"

#Calculating DO orbitals
dirlist = []
Nbf = 519
Nocc = 51
channel = 'beta'
cores = [1, 2]
nstate = 10
with open(masterDataFileName + "/dirlist_all", 'r') as dFile:
    for line in dFile:
        dirlist.append(line.split()[0])
    for state in range(nstate):
        print("Calculating DO for state: ", str(state+1))
        for d in dirlist:
            print(d)
            for coreLabel in cores:
                if coreLabel == 1:
                    captionFolder = "Cation_O"
                    coreOrb = 1
                if coreLabel == 2:
                    captionFolder = "Cation_N"
                    coreOrb = 2
                neutralFile = masterDataFileName + '/Neutral/' + d + '/tddft.out'
                cationFile = masterDataFileName + '/' + captionFolder + '/' + d + '/tddft.out'
                outputFile = "examples/HBQ/output"
                iNeutral = 1
                iCation = nstate
                bAO = calc_DO_single_state(neutralFile, cationFile, Nbf, Nocc, iNeutral, state + 1, channel)
                np.savetxt(outputFile + '/state_' + str(state + 1) + '/DO_' + channel + '_state' + str(state + 1) + '_core' + str(coreOrb) + '.dat', bAO, fmt = '%.6f')





Calculating DO for state:  1
step_01_0
Calculating DO for root 1
And the norm is: 0.0591003293225366
done in 0.8725030422210693 seconds!
Calculating DO for root 1
And the norm is: 0.080656463579403
done in 0.8880739212036133 seconds!
Calculating DO for state:  2
step_01_0
Calculating DO for root 2
And the norm is: 0.044525028324343484
done in 0.8780820369720459 seconds!
Calculating DO for root 2
And the norm is: 0.35697711377382974
done in 1.0088539123535156 seconds!
Calculating DO for state:  3
step_01_0
Calculating DO for root 3
And the norm is: 0.5395674696884273
done in 0.9321551322937012 seconds!
Calculating DO for root 3
And the norm is: 0.5427645176934008
done in 0.9997632503509521 seconds!
Calculating DO for state:  4
step_01_0
Calculating DO for root 4
And the norm is: 0.04995941185370866
done in 0.8604791164398193 seconds!
Calculating DO for root 4
And the norm is: 0.0
done in 0.7620739936828613 seconds!
Calculating DO for state:  5
step_01_0
Calculating DO for root 5
And the

In [16]:
import os
os.chdir("/Users/victormanuelfreixaslemus/Desktop/Projects/Git_portafolio/trXPES/tr-XPES")
from src.make_whole_xyz import make_whole_xyz
from src.make_case0 import make_case0
from src.get_MO_energy import get_MO_energy
import glob
import numpy as np
import matlab.engine

nstate = 10
masterDataFileName = "examples/HBQ/tr_001/"
outputFileName = "examples/HBQ/output/"

# Creating wholeCoords.xyz file
make_whole_xyz(masterDataFileName, outputFileName)
xyzFileName = outputFileName + "/wholeCoords.xyz"

# Creating the "case0.m" file with the trajectories information in Matlab format:
basis = 'def2-TZVP' # Basis name
n0 = 1 - 1 # Index of the first coordinates read from the ".xyz"
ntr = 1 # Index of the final coordinates read from the ".xyz"
nsteps = 1 # Number of structures to skip between "n0" and "ntr"
outFileName = 'transition_moments/case0.m' # Name of the ".mat" output file
testDysonFileName = 'transition_moments/testdyson.mat'
make_case0(xyzFileName, basis, n0, ntr, nsteps, outFileName, testDysonFileName)

dirlist = []
with open(masterDataFileName + '/dirlist_all', 'r') as dFile:
    for line in dFile:
        dirlist.append(line.split()[0])

# Calculating transition moments and averaging (sing the Matlab engine)
eng = matlab.engine.start_matlab()
# ***
matlabFolder = 'transition_moments'
# ***
eng.addpath(matlabFolder, nargout=0)
eng.case0(nargout = 0)
inputFileName = testDysonFileName
channel = 'beta'
e0 = 0.1  #eV 
de = 0.1  #eV 
ef = 16.0 #eV 
nGrid = 74        # Lebedev grid
w = np.zeros(nGrid, dtype = float)
nEnergy = 160     # Energy points
dE = 0.1          # Energy steps between points (eV)
hartree = 27.2114 #eV
with open('transition_moments/lebedevweightlist74.txt', 'r') as wFile:
    i = -1
    for line in wFile:
        i += 1
        w[i] = float(line.split()[0])
for d in dirlist:
    r = int(d.split('_')[1])
    outputDir = outputFileName + '/Dipole_results'
    #[os.remove(os.path.join(outputDir, f)) for f in os.listdir(outputDir) if os.path.isfile(os.path.join(outputDir, f))]
    print('Working on core: ' + str(r))
    integralDir = outputFileName + '/dysonorbint'
    e0 = 0.1  #eV 
    de = 0.1  #eV 
    ef = 16.0 #eV 
    eng.test_dysonint(inputFileName, r, integralDir, e0, ef, de, nargout = 0)
    print('test_dysonint done')
    orbIntDir = integralDir
    for state in range(1, nstate + 1):
        print('Working on state: ' + str(state))
        stateFile = 'state_' + str(state)
        DOFileName = outputFileName + '/' + stateFile + '/DO_' + channel + '_' + stateFile + '_' + 'core'
        nDO = 2
        e0 = 0.1  #eV 
        de = 0.1  #eV 
        ef = 16.0 #eV 
        eng.calc_dysondipintloop_lebedev74rod(orbIntDir, DOFileName, nDO, outputDir, e0, ef, de, r, nargout = 0)
        print('Dipole calculation done')
        # Calculating the signal from the dipoles
        eO = get_MO_energy(masterDataFileName + '/Neutral/' + d + '/tddft.out', 1) * hartree
        eN = get_MO_energy(masterDataFileName + '/Neutral/' + d + '/tddft.out', 2) * hartree
        for coreLabel in cores:
            if coreLabel == 1:
                e0 = eO
            if coreLabel == 2:
                e0 = eN
            signal = np.zeros((nEnergy, 2), dtype = float)
            for i in range(nGrid):
                with open(masterDataFileName + '/Dipole_results/dipolesIm' + str(i + 1) + '_' + str(coreLabel) + '_' + str(r) + '.dat', 'r') as dFile:
                    j = -1
                    for line in dFile:
                        j += 1
                        signal[j, 0] = 0.1 + j * dE
                        signal[j, 1] += w[i] * (float(line.split(',')[0])**2 + float(line.split(',')[1])**2 + float(line.split(',')[2])**2)
                with open(masterDataFileName + '/Dipole_results/dipolesRe' + str(i + 1) + '_' + str(coreLabel) + '_' + str(r) + '.dat', 'r') as dFile:
                    j = -1
                    for line in dFile:
                        j += 1
                        signal[j, 1] += w[i] * (float(line.split(',')[0])**2 + float(line.split(',')[1])**2 + float(line.split(',')[2])**2)
            with open(masterDataFileName + '/td_results_' + str(state) + '/signal_' + str(coreLabel) + '_' + d + '.dat', 'w') as sFile:
                for pair in signal:
                    sFile.write(str(pair[0] - e0) + '\t' + str(pair[1]) + '\n')
[os.remove(f) for f in glob.glob("dysonorbint/*")]



Working on core: 1

igrid =

     1


igrid =

     2


igrid =

     3


igrid =

     4


igrid =

     5


igrid =

     6


igrid =

     7


igrid =

     8


igrid =

     9


igrid =

    10


igrid =

    11


igrid =

    12


igrid =

    13


igrid =

    14


igrid =

    15


igrid =

    16


igrid =

    17


igrid =

    18


igrid =

    19


igrid =

    20


igrid =

    21


igrid =

    22


igrid =

    23


igrid =

    24


igrid =

    25


igrid =

    26


igrid =

    27


igrid =

    28


igrid =

    29


igrid =

    30


igrid =

    31


igrid =

    32


igrid =

    33


igrid =

    34


igrid =

    35


igrid =

    36


igrid =

    37


igrid =

    38


igrid =

    39


igrid =

    40


igrid =

    41


igrid =

    42


igrid =

    43


igrid =

    44


igrid =

    45


igrid =

    46


igrid =

    47


igrid =

    48


igrid =

    49


igrid =

    50


igrid =

    51


igrid =

    52


igrid =

    53


igrid =

    54


igrid =


Error using importdata (line 71)
Unable to open file.

Error in calc_dysondipintloop_lebedev74rod (line 77)
        realtemp = importdata(inputnamereal); 
                   ^^^^^^^^^^^^^^^^^^^^^^^^^



MatlabExecutionError: 
  File /Applications/MATLAB_R2024b.app/toolbox/matlab/iofun/importdata.m, line 71, in importdata

  File /Users/victormanuelfreixaslemus/Desktop/Projects/Git_portafolio/trXPES/tr-XPES/transition_moments/calc_dysondipintloop_lebedev74rod.m, line 77, in calc_dysondipintloop_lebedev74rod
Unable to open file.
